In [1]:
#!pip install skforecast
# Data processing
# ==============================================================================
import os
import pandas as pd
import numpy as np

# Plotting
# ==============================================================================
import matplotlib.pyplot as plt
from skforecast.plot import set_dark_theme
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as poff
pio.templates.default = "seaborn"
poff.init_notebook_mode(connected=True)

# Keras
# ==============================================================================
import keras
from keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.callbacks import EarlyStopping

# Time series modeling
# ==============================================================================
import skforecast
from skforecast.deep_learning import ForecasterRnn
from skforecast.deep_learning.utils import create_and_compile_model
from sklearn.preprocessing import MinMaxScaler
from skforecast.model_selection import TimeSeriesFold,  OneStepAheadFold
from skforecast.model_selection import backtesting_forecaster_multiseries
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

import sklearn
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFECV
from skforecast.recursive import ForecasterEquivalentDate
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect
from skforecast.model_selection import bayesian_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.feature_selection import select_features
from skforecast.model_selection import TimeSeriesFold
from skforecast.preprocessing import RollingFeatures

2024-12-29 21:34:59.748927: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df1 = pd.read_excel('/Users/christianhellum/Cand. Merc./Data-Science-Project/data_science_project/Praktik/data_final.xlsx')

In [3]:
df = df1.copy()
#drop missing data
df = df.dropna()

In [4]:
# Initially include all rows
df["Include"] = True

# Function to tag rows based on conditions
def tag_abnormal_values(df, condition, include_col='Include'):
    """
    Updates the Include column to False for rows that satisfy the given condition.

    :param df: The DataFrame to modify.
    :param condition: A condition (boolean array or series) for abnormal values.
    :param include_col: The column name to update for inclusion.
    :return: Modified DataFrame with updated Include column.
    """
    df.loc[condition, include_col] = False
    return df

# Define conditions for abnormality
conditions = [
    (df['energy_total'] < 1) | (df['energy_total'] > 2800)
]

# Exclude abnormal features
for condition in conditions:
    df = tag_abnormal_values(df, condition)
# Cut out all abnormal values
df = df[df.Include].drop(["Include"], axis=1)

#delete columns 'dhi', 'dni', 'precip', 'snow', 'wind_spd'
df = df.drop(['dhi', 'dni', 'precip', 'snow', 'wind_spd'], axis=1)

#extract the hour from the datetime column
df['hour'] = pd.DatetimeIndex(df['datetime']).hour

#only keep rows where hour is between 8 and 18
df = df[(df['hour'] >= 8) & (df['hour'] <= 18)]
#delete three first rows
df = df.iloc[3:]

#drop hour column
df = df.drop(['hour'], axis=1)
df = df.drop(['hour_nr'], axis=1)
df = df.drop(['day_of_year'], axis=1)
df = df.drop(['month_nr'], axis=1)

#start the data from date 2022-03-30 08:00:00
df = df[df['datetime'] >= '2022-03-30 08:00:00']

#delete energy_total in df
df = df.drop(columns=['datetime'], axis=1)

In [5]:
import pandas as pd

# Define columns and window periods
columns_to_window = [
    'clouds', 'ghi', 'pres', 'slp', 'solar_rad', 'temp', 'uv', 
    'month_sin', 'month_cos', 'day_sin', 'day_cos', 
    'hour_sin', 'hour_cos', 'sine_elevation', 'cosine_elevation',
    'sine_azimuth', 'cosine_azimuth'
]
window_periods = [10, 20, 30]

def create_window_features(df, columns, periods):
    """
    Create windowed features for specified columns over given time periods.
    
    Args:
        df (pd.DataFrame): Original DataFrame.
        columns (list): List of column names to create windowed features for.
        periods (list): List of time periods for the windows.
        
    Returns:
        pd.DataFrame: DataFrame with windowed features.
    """
    for col in columns:
        for period in periods:
            df[f'{col}_lag_{period}'] = df[col].shift(period)
    return df

# Assuming `df` is your DataFrame
df = create_window_features(df, columns_to_window, window_periods)

# Drop rows with NaN values caused by shifting
df = df.dropna()

# Display the first few rows of the updated DataFrame
print(df.head())

     clouds  ghi  pres   slp  solar_rad  temp   uv  energy_total  month_sin  \
175     100  226  1013  1016         91   3.9  0.6        596.28   0.866025   
176     100   87  1013  1016         54   2.9  0.6        260.92   0.866025   
177      97    0  1014  1017          0   1.8  0.0        168.04   0.866025   
191       0  393  1017  1020        393   2.3  2.3        222.96   0.866025   
192      54  509  1018  1021        453   2.5  2.6        541.52   0.866025   

     month_cos  ...  sine_elevation_lag_30  cosine_elevation_lag_10  \
175       -0.5  ...               0.406737                 0.992115   
176       -0.5  ...               0.509041                 0.999781   
177       -0.5  ...               0.579281                 0.907777   
191       -0.5  ...               0.614285                 0.853551   
192       -0.5  ...               0.611527                 0.806960   

     cosine_elevation_lag_20  cosine_elevation_lag_30  sine_azimuth_lag_10  \
175                 

In [6]:
#box cox transformation of energy_total
from scipy.stats import boxcox
df['energy_total'], lmbda = boxcox(df['energy_total'])

In [7]:
df.reset_index(drop=True, inplace=True)  # Use RangeIndex

train_end = int(0.75 * len(df))
val_end = train_end + int(0.15 * len(df))

df_train = df.iloc[:train_end, :].copy()
df_val = df.iloc[train_end:val_end, :].copy()
df_test = df.iloc[val_end:, :].copy()

print(f"Train size: {len(df_train)}")
print(f"Validation size: {len(df_val)}")
print(f"Test size: {len(df_test)}")

Train size: 7299
Validation size: 1459
Test size: 975


In [8]:
# Calculate mean and standard deviation for all features except 'energy_total'
features_to_scale = [col for col in df_train.columns if col != 'energy_total']
train_mean = df_train[features_to_scale].mean()
train_std = df_train[features_to_scale].std()

# Scale the selected features
df_train[features_to_scale] = (df_train[features_to_scale] - train_mean) / train_std
df_val[features_to_scale] = (df_val[features_to_scale] - train_mean) / train_std
df_test[features_to_scale] = (df_test[features_to_scale] - train_mean) / train_std
df[features_to_scale] = (df[features_to_scale] - train_mean) / train_std

In [38]:
window_features = RollingFeatures(stats=["mean"], window_sizes=10 * 3)

In [39]:
# Folds used for the hyperparameter search and backtesting
# ==============================================================================
cv_search = OneStepAheadFold(initial_train_size = len(df_train))

cv_backtesting = TimeSeriesFold(
                    steps              = 10,
                    initial_train_size = len(df[:val_end]),
                    refit              = False,
                 )

In [40]:
#define that exog_features are all columns except energy_total
exog_features = [col for col in df.columns if col != 'energy_total']

In [43]:
from xgboost import XGBRegressor
# Create forecaster
# ==============================================================================
params_xgb = {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 500}
forecaster = ForecasterRecursive(
                 regressor = XGBRegressor(random_state=123, **params_xgb),
                 lags = 30,
                 window_features=window_features
             )

In [44]:
# Backtesting model with exogenous variables on test data
# ==============================================================================
metrics, predictions = backtesting_forecaster(
    forecaster    = forecaster,
    y             = df['energy_total'],
    exog          = df[exog_features],
    cv            = cv_backtesting,
    metric        = 'mean_absolute_error',
    n_jobs        = 'auto',
    verbose       = False,
)
metrics

  0%|          | 0/98 [00:00<?, ?it/s]

,mean_absolute_error
0,5.209905


In [45]:
from sklearn.linear_model import Ridge
# Create forecaster
# ==============================================================================
params_ridge = {'alpha': 0.001}
forecaster = forecaster = ForecasterRecursive(
                 regressor = Ridge(random_state=123, **params_ridge),
                 lags = 30,
                 window_features=window_features
             )

In [46]:
# Backtesting model with exogenous variables on test data
# ==============================================================================
metrics, predictions = backtesting_forecaster(
    forecaster    = forecaster,
    y             = df['energy_total'],
    exog          = df[exog_features],
    cv            = cv_backtesting,
    metric        = 'mean_absolute_error',
    n_jobs        = 'auto',
    verbose       = False,
)
metrics

  0%|          | 0/98 [00:00<?, ?it/s]

,mean_absolute_error
0,4.914949


In [47]:
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection  import KFold
# Create stacking regressor
# ==============================================================================
estimators = [
    ('ridge', Ridge(random_state=123, **params_ridge)),
    ('xgb', XGBRegressor(random_state=123, **params_xgb)),
]
stacking_regressor = StackingRegressor(
                        estimators = estimators,
                        final_estimator = Ridge(),
                        cv = KFold(n_splits=10, shuffle=False)
                    )
stacking_regressor

StackingRegressor(cv=KFold(n_splits=10, random_state=None, shuffle=False),
                  estimators=[('ridge', Ridge(alpha=0.001, random_state=123)),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_...
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.1, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=5,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=500, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=123, ...))],
                  final_estimator=Ridge())

In [48]:
# Create forecaster
# ==============================================================================
forecaster = ForecasterRecursive(
                 regressor = stacking_regressor,
                 lags = 30
             )

In [49]:
# Backtesting model with exogenous variables on test data
# ==============================================================================
metrics, predictions = backtesting_forecaster(
    forecaster    = forecaster,
    y             = df['energy_total'],
    exog          = df[exog_features],
    cv            = cv_backtesting,
    metric        = 'mean_absolute_error',
    n_jobs        = 'auto',
    verbose       = False,
)
metrics

  0%|          | 0/98 [00:00<?, ?it/s]

,mean_absolute_error
0,4.802357


In [52]:
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

# Grid search of hyperparameters and lags
# ==============================================================================
param_grid = {
    'ridge__alpha': [0.001, 0.01, 0.1, 1, 10],
    'xgb__n_estimators': [100, 500],
    'xgb__max_depth': [3, 5, 10],
    'xgb__learning_rate': [0.01, 0.1],
    'xgb__subsample': [0.1, 1],
    'xgb__colsample_bytree': [0.1, 1],
    'xgb__gamma': [0, 1],
    'xgb__reg_alpha': [0, 1],
    'xgb__reg_lambda': [0, 1]}

# Lags used as predictors
lags_grid = [30]

results_grid = grid_search_forecaster(
                   forecaster         = forecaster,
                   y                  = df.loc[:val_end, 'energy_total'],
                   exog               = df.loc[:val_end, exog_features],
                   param_grid         = param_grid,
                   lags_grid          = lags_grid,
                   metric             = 'mean_squared_error',
                   return_best        = True,
                    cv=TimeSeriesFold(steps=10, initial_train_size=len(df[:val_end]) - 10),
                   n_jobs             = 'auto',
                   verbose            = False
               )

results_grid.head()

lags grid:   0%|          | 0/1 [00:00<?, ?it/s]

params grid:   0%|          | 0/1920 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [15]:
from xgboost import XGBRegressor
# Create forecaster
# ==============================================================================
forecaster = ForecasterRecursive(
    regressor = XGBRegressor(tree_method='hist', enable_categorical=True, random_state=123),
    lags = 30,
    window_features  = window_features
)

In [16]:
# Hyperparameters search
# ==============================================================================
# Lags grid
lags_grid = [10, 20, 30]

# Regressor hyperparameters search space
def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 400, 1200, step=100),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 1),
        'subsample'       : trial.suggest_float('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'gamma'           : trial.suggest_float('gamma', 0, 1),
        'reg_alpha'       : trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda'      : trial.suggest_float('reg_lambda', 0, 1),
        'lags'            : trial.suggest_categorical('lags', lags_grid)
    } 
    return search_space

results_search, frozen_trial = bayesian_search_forecaster(
    forecaster    = forecaster,
    y             = df.loc[:val_end, 'energy_total'],
    exog          = df.loc[:val_end, exog_features],
    cv            = cv_search,
    search_space  = search_space,
    metric        = 'mean_absolute_error',
    n_trials      = 20,
    random_state  = 123,
    return_best   = True,
    n_jobs        = 'auto',
    verbose       = False,
    show_progress = True
)

/Users/christianhellum/Library/Caches/pypoetry/virtualenvs/data-science-project-PFmYB0_u-py3.12/lib/python3.12/site-packages/skforecast/model_selection/_search.py:715: OneStepAheadValidationWarning:

One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=OneStepAheadValidationWarning)



  0%|          | 0/20 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30] 
  Parameters: {'n_estimators': 1200, 'max_depth': 9, 'learning_rate': 0.011347295681454445, 'subsample': 0.26231918597596127, 'colsample_bytree': 0.8198528183931876, 'gamma': 0.01151679189950039, 'reg_alpha': 0.18112786809839324, 'reg_lambda': 0.32121257564389666}
  One-step-ahead metric: 3.585534051828021


In [17]:
# Backtesting model with exogenous variables on test data
# ==============================================================================
metric_xgboost, predictions = backtesting_forecaster(
    forecaster    = forecaster,
    y             = df['energy_total'],
    exog          = df[exog_features],
    cv            = cv_backtesting,
    metric        = 'mean_absolute_error',
    n_jobs        = 'auto',
    verbose       = False,
)
metric_xgboost

  0%|          | 0/98 [00:00<?, ?it/s]

/Users/christianhellum/Library/Caches/pypoetry/virtualenvs/data-science-project-PFmYB0_u-py3.12/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning:

This process (pid=14350) is multi-threaded, use of fork() may lead to deadlocks in the child.



,mean_absolute_error
0,5.060813
